In [1]:
"""https://www.kaggle.com/wspinkaggle/rps-basic-decision-tree"""

from kaggle_environments import make
from kaggle_environments.envs.rps.agents import agents

print(agents)

Loading environment football failed: No module named 'gfootball'
{'rock': <function rock at 0x7f9588073940>, 'paper': <function paper at 0x7f9572ea48b0>, 'scissors': <function scissors at 0x7f9572ea49d0>, 'copy_opponent': <function copy_opponent at 0x7f9572ea4a60>, 'reactionary': <function reactionary at 0x7f9572ea4af0>, 'counter_reactionary': <function counter_reactionary at 0x7f9572ea4b80>, 'statistical': <function statistical at 0x7f9572ea4c10>}


In [2]:
# adding random agent
import random

def random_agent(observation, configuration):
    return random.randint(0, configuration.signs-1)

In [3]:
%%writefile random_forest.py

import random
import numpy as np

from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier

sample_size = 5
opponent_observations = []
my_actions = []

def make_train_set(opponent_observations, my_actions, sample_size):
    observed = np.array([list(x) for x in zip(opponent_observations, my_actions)])
    prediction_sample = observed[-sample_size:].reshape(1, sample_size * 2)

    splits = TimeSeriesSplit(
        max_train_size=sample_size,
        n_splits=len(observed) - sample_size
    )
        
    X = np.array([
            observed[train_index] for train_index, _ in splits.split(observed)
    ])
    
    X = X.reshape(X.shape[0], X.shape[1] * X.shape[2])

    Y = np.array([
            observed[y_index][0][0] for _, y_index in splits.split(observed)
    ])

    Y = Y.reshape(Y.shape[0], 1)
    
    return X, Y, prediction_sample

def action_decision_tree_dev(opponent_observations, my_actions,
    sample_size, configuration=None):

    X, Y, prediction_sample = make_train_set(opponent_observations, my_actions, sample_size)

    model = RandomForestClassifier(
        n_estimators=100,
        criterion="entropy",
        max_depth = 4,
    )

    model.fit(X, Y)

    prediction = model.predict(prediction_sample)
    
    # action is what beats the opponents prediciton
    action = (prediction + 1) % (configuration.signs if configuration else 3)
    action = int(action[0])
    
    # check if fails on long wait
    import time
    
    return action

def random_forest_agent(observation, configuration):
    global opponent_observations
    global my_actions
    global sample_size
    
    if observation.step == 0:
        action = 1
        my_actions.append(action)
        return action
    
    elif observation.step <= 2 * sample_size:
        opponent_observations.append(observation.lastOpponentAction)
        action = random.randint(0, configuration.signs-1)
        my_actions.append(action)
        return action
        
    else:
        opponent_observations.append(observation.lastOpponentAction)
        
        action = action_decision_tree_dev(
            opponent_observations,
            my_actions,
            sample_size,
            configuration
        )
        
        my_actions.append(action)
        return action

Overwriting random_forest.py


In [4]:
%%writefile xgboost_agent.py

import random
import numpy as np

from sklearn.model_selection import TimeSeriesSplit
from xgboost import XGBClassifier

sample_size = 5
opponent_observations = []
my_actions = []

def make_train_set(opponent_observations, my_actions, sample_size):
    observed = np.array([list(x) for x in zip(opponent_observations, my_actions)])
    prediction_sample = observed[-sample_size:].reshape(1, sample_size * 2)

    splits = TimeSeriesSplit(
        max_train_size=sample_size,
        n_splits=len(observed) - sample_size
    )
        
    X = np.array([
            observed[train_index] for train_index, _ in splits.split(observed)
    ])
    
    X = X.reshape(X.shape[0], X.shape[1] * X.shape[2])

    Y = np.array([
            observed[y_index][0][0] for _, y_index in splits.split(observed)
    ])

    Y = Y.reshape(Y.shape[0], 1)
    
    return X, Y, prediction_sample

def action_decision_tree_dev(opponent_observations, my_actions,
    sample_size, configuration=None):

    X, Y, prediction_sample = make_train_set(opponent_observations, my_actions, sample_size)

    model = XGBClassifier(
        n_estimators=100,
        criterion="entropy",
        max_depth = 4,
    )

    model.fit(X, Y)

    prediction = model.predict(prediction_sample)
    
    # action is what beats the opponents prediciton
    action = (prediction + 1) % (configuration.signs if configuration else 3)
    action = int(action[0])
    
    # check if fails on long wait
    import time
    
    return action

def xgboost_agent(observation, configuration):
    global opponent_observations
    global my_actions
    global sample_size
    
    if observation.step == 0:
        action = 1
        my_actions.append(action)
        return action
    
    elif observation.step <= 2 * sample_size:
        opponent_observations.append(observation.lastOpponentAction)
        action = random.randint(0, configuration.signs-1)
        my_actions.append(action)
        return action
        
    else:
        opponent_observations.append(observation.lastOpponentAction)
        
        action = action_decision_tree_dev(
            opponent_observations,
            my_actions,
            sample_size,
            configuration
        )
        
        my_actions.append(action)
        return action

Overwriting xgboost_agent.py


In [5]:
%%writefile adaboost_agent.py

import random
import numpy as np

from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import AdaBoostClassifier

sample_size = 5
opponent_observations = []
my_actions = []

def make_train_set(opponent_observations, my_actions, sample_size):
    observed = np.array([list(x) for x in zip(opponent_observations, my_actions)])
    prediction_sample = observed[-sample_size:].reshape(1, sample_size * 2)

    splits = TimeSeriesSplit(
        max_train_size=sample_size,
        n_splits=len(observed) - sample_size
    )
        
    X = np.array([
            observed[train_index] for train_index, _ in splits.split(observed)
    ])
    
    X = X.reshape(X.shape[0], X.shape[1] * X.shape[2])

    Y = np.array([
            observed[y_index][0][0] for _, y_index in splits.split(observed)
    ])

    Y = Y.reshape(Y.shape[0], 1)
    
    return X, Y, prediction_sample

def action_decision_tree_dev(opponent_observations, my_actions,
    sample_size, configuration=None):

    X, Y, prediction_sample = make_train_set(opponent_observations, my_actions, sample_size)

    model = AdaBoostClassifier(
        n_estimators=50,
        learning_rate=1.0
    )

    model.fit(X, Y)

    prediction = model.predict(prediction_sample)
    
    # action is what beats the opponents prediciton
    action = (prediction + 1) % (configuration.signs if configuration else 3)
    action = int(action[0])
    
    # check if fails on long wait
    import time
    
    return action

def adaboost_agent(observation, configuration):
    global opponent_observations
    global my_actions
    global sample_size
    
    if observation.step == 0:
        action = 1
        my_actions.append(action)
        return action
    
    elif observation.step <= 2 * sample_size:
        opponent_observations.append(observation.lastOpponentAction)
        action = random.randint(0, configuration.signs-1)
        my_actions.append(action)
        return action
        
    else:
        opponent_observations.append(observation.lastOpponentAction)
        
        action = action_decision_tree_dev(
            opponent_observations,
            my_actions,
            sample_size,
            configuration
        )
        
        my_actions.append(action)
        return action

Overwriting adaboost_agent.py


In [6]:
#%run -i "random_forest.py"

from random_forest import random_forest_agent
from xgboost_agent import xgboost_agent
from adaboost_agent import adaboost_agent

In [7]:
env = make("rps", configuration={"episodeSteps": 100}, debug=False)

env.run([random_forest_agent, agents["statistical"]])
env.render(mode="ipython", width=600, height=600)

In [8]:
env = make("rps", configuration={"episodeSteps": 100}, debug=False)

env.run([xgboost_agent, agents["statistical"]])
env.render(mode="ipython", width=600, height=600)

In [9]:
env = make("rps", configuration={"episodeSteps": 100}, debug=False)

env.run([xgboost_agent, random_forest_agent])
env.render(mode="ipython", width=600, height=600)

In [10]:
env = make("rps", configuration={"episodeSteps": 100}, debug=True)

env.run([adaboost_agent, agents["statistical"]])
env.render(mode="ipython", width=600, height=600)

/home/wolle/miniconda3/envs/data_science/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/wolle/miniconda3/envs/data_science/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/wolle/miniconda3/envs/data_science/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/wolle/miniconda3/envs/data_science/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d arr

/home/wolle/miniconda3/envs/data_science/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/wolle/miniconda3/envs/data_science/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/wolle/miniconda3/envs/data_science/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/wolle/miniconda3/envs/data_science/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d arr

/home/wolle/miniconda3/envs/data_science/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/wolle/miniconda3/envs/data_science/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/wolle/miniconda3/envs/data_science/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/wolle/miniconda3/envs/data_science/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d arr

env = make("rps", configuration={"episodeSteps": 100}, debug=True)

env.run([tree_agent, agents["counter_reactionary"]])
env.render(mode="ipython", width=600, height=600)